In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. System Parameters (LQG Components)
A = np.array([[1.0]]) # Simple 1D dynamics
B = np.array([[1.0]])
C = np.array([[1.0]]) # Measurement mapping
Q = 0.01 # Process noise covariance (Sigma_w)
R = 0.1  # Measurement noise covariance (Sigma_v)

# 2. Kalman Filter Implementation (State Estimation)
def kalman_filter_step(s_hat, sigma, a, y):
    """
    Implements a single Predict/Update cycle of the Kalman Filter.
    Reference: Stanford CS229 Lecture 19.
    """
    # PREDICT: s_hat_{t+1|t}
    s_hat_pred = A.dot(s_hat) + B.dot(a)
    sigma_pred = A.dot(sigma).dot(A.T) + Q
    
    # UPDATE: Incorporate observation y
    # Kalman Gain K
    K = sigma_pred.dot(C.T).dot(np.linalg.inv(C.dot(sigma_pred).dot(C.T) + R))
    
    # Final state estimate s_hat_{t+1|t+1}
    s_hat_new = s_hat_pred + K.dot(y - C.dot(s_hat_pred))
    sigma_new = (np.eye(len(s_hat)) - K.dot(C)).dot(sigma_pred)
    
    return s_hat_new, sigma_new

# 3. Execution on Noisy Data
data = pd.read_csv('data/noisy_sensors.csv')
s_hat = np.array([[0.0]]) # Initial estimate
sigma = np.array([[1.0]]) # Initial uncertainty

estimates = []
for i in range(len(data)):
    s_hat, sigma = kalman_filter_step(s_hat, sigma, data['action'][i], data['obs_pos'][i])
    estimates.append(s_hat[0, 0])

# 4. Visualization
plt.plot(data['true_pos'], label='True Position', linestyle='--')
plt.plot(data['obs_pos'], label='Noisy Observation', alpha=0.5)
plt.plot(estimates, label='Kalman Estimate (LQG Head)', color='red')
plt.legend()
plt.title('Autonomous State Estimation: Kalman Filter')
plt.show()